# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

$
p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T \\
A = ((1-\alpha)M + \frac{\alpha}{N}E) \\
Opakujte: \\
\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}
$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [1]:
import numpy as np
from graph import Graph

In [2]:
def read_data_to_graph(filename: str) -> Graph:
    try:
        with open(filename, 'r') as data_file: 
            edges = Graph.read_edges_from_file(data_file)
            new_graph = Graph(edges)
            return new_graph
    except FileNotFoundError as e:
        print(f"ERROR while reading: {e}")
        return

In [3]:
filename = 'data/graph_1.txt'
graph = read_data_to_graph(filename)
print(graph.adjacency_matrix)
print()
print(graph.prob_transition_matrix)

[[0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]


### Načítání vstupů (grafů) z dat.

In [4]:
filenames = [
    'data/graph_1.txt',
    'data/graph_2.txt',
    'data/graph_3.txt',
    'data/graph_4.txt',
    'data/graph_5.txt',
    'data/graph_6.txt',
]
graphs = list()
for i in range(len(filenames)):
    graphs.append(read_data_to_graph(filename=filenames[i]))

### Implementace PageRank

In [15]:
iters = 100

for n, graph in enumerate(graphs):
    N = graph.prob_transition_matrix.shape[0]  # number of pages
    pageranks = np.ones(N) / N
    alpha = np.full((N, N), 0.2)  # define alpha as NxN values

    M = graph.prob_transition_matrix
    
    # Find the dangling nodes (pages with no path to others => zeros on its matrix row)
    dangling_nodes = np.sum(M, axis=1) == 0
    # Set alpha for this page to 1
    for i in range(N):
        if dangling_nodes[i]:
            alpha[i, :] = 1

    E = np.ones((N, N))
    A = (1 - alpha) * graph.prob_transition_matrix + alpha / N * E

    # Update pageranks
    for _ in range(iters):
        pageranks = A.T @ pageranks


    print(filenames[n][5:] + ':')
    print(pageranks)
    print('-' * 80)

graph_1.txt:
[0.0656044  0.11808792 0.16007474 0.19366419 0.22053575 0.242033  ]
--------------------------------------------------------------------------------
graph_2.txt:
[0.2 0.2 0.2 0.2 0.2]
--------------------------------------------------------------------------------
graph_3.txt:
[0.17857143 0.32142857 0.32142857 0.17857143]
--------------------------------------------------------------------------------
graph_4.txt:
[0.27257372 0.15666713 0.13837881 0.10924643 0.18531604 0.06563464
 0.07218322]
--------------------------------------------------------------------------------
graph_5.txt:
[0.00163995 0.00163995 0.00163995 0.00163995 0.00163995 0.00185861
 0.00177115 0.00182737 0.00196794 0.00196794 0.00182737 0.00177115
 0.00196794 0.00196794 0.00177115 0.00169711 0.00170178 0.0019428
 0.00198014 0.00169465 0.00365343 0.003554   0.00169465 0.00402315
 0.00182136 0.00185236 0.00187664 0.00170178 0.00170178 0.00240599
 0.00197445 0.00175037 0.00246671 0.00225534 0.00253521 0.001

### Předpokládaný výstup

#### graph_1.txt
```python
array([[0.0656044 ],
       [0.11808792],
       [0.16007474],
       [0.19366419],
       [0.22053575],
       [0.242033  ]])
```
       
#### graph_2.txt
```python
array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])
```


#### graph_3.txt
```python
array([[0.17857143],
       [0.32142857],
       [0.32142857],
       [0.17857143]])
```


#### graph_4.txt
```python
array([[0.27257372],
       [0.15666713],
       [0.13837881],
       [0.10924643],
       [0.18531604],
       [0.06563464],
       [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [18]:
iters = 100

for n, graph in enumerate(graphs):
    A = graph.adjacency_matrix
    N = A.shape[0]
    a = np.ones(N)  # Authority
    h = np.ones(N)  # Hub Scores

    # HITS algorithm
    for i in range(iters):
        # Update hub scores
        h_new = A @ a
        h_new /= np.sum(np.abs(h_new))  # Normalize h_new with L1 norm
        
        # Update authority scores
        a_new = A.T @ h_new
        a_new /= np.sum(np.abs(a_new))  # Normalize a_new with L1 norm

        h, a = h_new, a_new
    
    print(filenames[n][5:] + ':')
    print(f"Authority: {a}")
    print(f"Hub: {h}")
    print('-' * 80)

graph_1.txt:
Authority: [0.  0.2 0.2 0.2 0.2 0.2]
Hub: [0.2 0.2 0.2 0.2 0.2 0. ]
--------------------------------------------------------------------------------
graph_2.txt:
Authority: [0.2 0.2 0.2 0.2 0.2]
Hub: [0.2 0.2 0.2 0.2 0.2]
--------------------------------------------------------------------------------
graph_3.txt:
Authority: [0.19098301 0.30901699 0.30901699 0.19098301]
Hub: [0.19098301 0.30901699 0.30901699 0.19098301]
--------------------------------------------------------------------------------
graph_4.txt:
Authority: [0.13948389 0.17791203 0.20082321 0.14017775 0.20142536 0.05608926
 0.08408849]
Hub: [0.27545318 0.04776231 0.10868324 0.19865956 0.1837346  0.11673471
 0.06897241]
--------------------------------------------------------------------------------
graph_5.txt:
Authority: [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 4.23652715e-112 1.53782147e-051 4.76937960e-091
 3.92246584e-170 3.92246584e-170 4.76937960e-091 1.5378214

### Předpokládaný výstup

#### graph_1.txt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

### Předpokládaný výstup

```python
print(inverted_index['be'])
print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.